# Transactions in SQLite

In this notebook, you'll learn about transactions in SQLite - how to group multiple operations into atomic units that either succeed completely or fail completely, maintaining database consistency.

In [ ]:
import sqlite3

# Connect to database
conn = sqlite3.connect('transactions_demo.db')
cursor = conn.cursor()

# Create tables for demonstration
cursor.execute('''
    CREATE TABLE accounts (
        id INTEGER PRIMARY KEY,
        name TEXT NOT NULL,
        balance REAL NOT NULL DEFAULT 0.0
    )
''')

cursor.execute('''
    CREATE TABLE transactions (
        id INTEGER PRIMARY KEY,
        from_account INTEGER,
        to_account INTEGER,
        amount REAL NOT NULL,
        timestamp DATETIME DEFAULT CURRENT_TIMESTAMP,
        FOREIGN KEY (from_account) REFERENCES accounts (id),
        FOREIGN KEY (to_account) REFERENCES accounts (id)
    )
''')

# Insert test accounts
accounts = [
    ('Alice', 1000.00),
    ('Bob', 500.00),
    ('Charlie', 200.00)
]

cursor.executemany('INSERT INTO accounts (name, balance) VALUES (?, ?)', accounts)

print('Database and test data created')

## What are Transactions?

A transaction is a sequence of SQL operations that are treated as a single unit. Transactions follow the ACID properties:

- **Atomicity**: All operations succeed or all fail
- **Consistency**: Database remains in a consistent state
- **Isolation**: Transactions don't interfere with each other
- **Durability**: Changes persist even after system failure

SQLite transactions are:
- **Atomic**: Either all changes are applied or none
- **Consistent**: Database constraints are maintained
- **Isolated**: Changes aren't visible to other connections until committed
- **Durable**: Changes survive power failures and crashes

## Auto-Commit Mode

By default, SQLite operates in auto-commit mode where each SQL statement is its own transaction.

In [ ]:
# Check current transaction state
print('In transaction:', conn.in_transaction)
print('Isolation level:', conn.isolation_level)

# Each statement is automatically committed
cursor.execute('UPDATE accounts SET balance = balance + 100 WHERE name = "Alice"')
print('Auto-committed update completed')

# Check balance
cursor.execute('SELECT balance FROM accounts WHERE name = "Alice"')
print('Alice balance:', cursor.fetchone()[0])

## Explicit Transactions

Use explicit transactions to group multiple operations:

In [ ]:
# Start a transaction
conn.execute('BEGIN TRANSACTION')

print('Transaction started')
print('In transaction:', conn.in_transaction)

# Perform multiple operations
cursor.execute('UPDATE accounts SET balance = balance - 200 WHERE name = "Alice"')
cursor.execute('UPDATE accounts SET balance = balance + 200 WHERE name = "Bob"')

print('Operations completed within transaction')

# Check balances (changes not visible to other connections yet)
cursor.execute('SELECT name, balance FROM accounts WHERE name IN ("Alice", "Bob")')
balances = cursor.fetchall()
print('Balances within transaction:', balances)

# Commit the transaction
conn.commit()
print('Transaction committed')
print('In transaction:', conn.in_transaction)

## Transfer Money Example

A classic example of why transactions are important - transferring money between accounts:

In [ ]:
def transfer_money(from_account, to_account, amount):
    """Transfer money between accounts using a transaction"""
    try:
        # Start transaction
        conn.execute('BEGIN TRANSACTION')
        
        # Check if sender has sufficient funds
        cursor.execute('SELECT balance FROM accounts WHERE name = ?', (from_account,))
        sender_balance = cursor.fetchone()[0]
        
        if sender_balance < amount:
            raise ValueError(f'Insufficient funds: {from_account} has {sender_balance}, needs {amount}')
        
        # Debit sender
        cursor.execute('UPDATE accounts SET balance = balance - ? WHERE name = ?', (amount, from_account))
        
        # Credit receiver
        cursor.execute('UPDATE accounts SET balance = balance + ? WHERE name = ?', (amount, to_account))
        
        # Record the transaction
        cursor.execute('INSERT INTO transactions (from_account, to_account, amount) VALUES (
            (SELECT id FROM accounts WHERE name = ?),
            (SELECT id FROM accounts WHERE name = ?),
            ?
        )', (from_account, to_account, amount))
        
        # Commit transaction
        conn.commit()
        print(f'Successfully transferred ${amount} from {from_account} to {to_account}')
        return True
        
    except Exception as e:
        # Rollback on error
        conn.rollback()
        print(f'Transfer failed: {e}')
        return False

print('Transfer function defined')

# Test successful transfer
transfer_money('Alice', 'Charlie', 150.00)

# Check balances
cursor.execute('SELECT name, balance FROM accounts ORDER BY name')
balances = cursor.fetchall()
print('Account balances:', balances)

## Rollback on Error

If an error occurs during a transaction, all changes are rolled back:

In [ ]:
# Attempt transfer that will fail
transfer_money('Bob', 'Alice', 1000.00)  # Bob only has 700

# Check that balances are unchanged
cursor.execute('SELECT name, balance FROM accounts ORDER BY name')
balances = cursor.fetchall()
print('Account balances after failed transfer:', balances)

# Check transaction records
cursor.execute('SELECT COUNT(*) FROM transactions')
transaction_count = cursor.fetchone()[0]
print(f'Transactions recorded: {transaction_count}')

## Using Context Managers

Python's `with` statement provides automatic transaction management:

In [ ]:
def transfer_with_context_manager(from_account, to_account, amount):
    """Transfer money using context manager for automatic transaction handling"""
    try:
        with conn:  # This starts a transaction
            # Check balance
            cursor.execute('SELECT balance FROM accounts WHERE name = ?', (from_account,))
            sender_balance = cursor.fetchone()[0]
            
            if sender_balance < amount:
                raise ValueError('Insufficient funds')
            
            # Perform transfer
            cursor.execute('UPDATE accounts SET balance = balance - ? WHERE name = ?', (amount, from_account))
            cursor.execute('UPDATE accounts SET balance = balance + ? WHERE name = ?', (amount, to_account))
            
            # Record transaction
            cursor.execute('''
                INSERT INTO transactions (from_account, to_account, amount)
                VALUES (
                    (SELECT id FROM accounts WHERE name = ?),
                    (SELECT id FROM accounts WHERE name = ?),
                    ?
                )
            ''', (from_account, to_account, amount))
            
        print(f'Context manager: Successfully transferred ${amount}')
        return True
        
    except Exception as e:
        print(f'Context manager: Transfer failed: {e}')
        return False

print('Context manager transfer function defined')

# Test context manager
transfer_with_context_manager('Alice', 'Bob', 50.00)

# Check final balances
cursor.execute('SELECT name, balance FROM accounts ORDER BY name')
balances = cursor.fetchall()
print('Final account balances:', balances)

## Transaction Isolation Levels

SQLite supports different isolation levels:

In [ ]:
# Check current isolation level
print('Current isolation level:', conn.isolation_level)

# Create a connection with no auto-commit (isolation_level=None)
conn_manual = sqlite3.connect('transactions_demo.db', isolation_level=None)
cursor_manual = conn_manual.cursor()

print('Manual commit connection created')
print('Isolation level:', conn_manual.isolation_level)

# With isolation_level=None, you must manually begin transactions
cursor_manual.execute('BEGIN')
cursor_manual.execute('UPDATE accounts SET balance = balance + 10 WHERE name = "Charlie"')
print('Update executed but not committed')

# Check if visible in original connection
cursor.execute('SELECT balance FROM accounts WHERE name = "Charlie"')
charlie_balance = cursor.fetchone()[0]
print(f'Charlie balance in original connection: {charlie_balance}')

# Commit in manual connection
conn_manual.commit()
print('Manual transaction committed')

# Now check again
cursor.execute('SELECT balance FROM accounts WHERE name = "Charlie"')
charlie_balance = cursor.fetchone()[0]
print(f'Charlie balance after commit: {charlie_balance}')

conn_manual.close()

## Savepoints

Savepoints allow partial rollback within a transaction:

In [ ]:
# Start transaction with savepoints
conn.execute('BEGIN')

# First operation
cursor.execute('UPDATE accounts SET balance = balance + 5 WHERE name = "Alice"')
cursor.execute('SAVEPOINT sp1')
print('Savepoint sp1 created')

# Second operation
cursor.execute('UPDATE accounts SET balance = balance + 5 WHERE name = "Bob"')
cursor.execute('SAVEPOINT sp2')
print('Savepoint sp2 created')

# Third operation that might fail
cursor.execute('UPDATE accounts SET balance = balance + 5 WHERE name = "NonExistent"')
print('Attempted update on non-existent account')

# Rollback to savepoint sp2
cursor.execute('ROLLBACK TO sp2')
print('Rolled back to savepoint sp2')

# Commit the transaction (only changes up to sp2)
conn.commit()
print('Transaction committed with partial rollback')

# Check final balances
cursor.execute('SELECT name, balance FROM accounts WHERE name IN ("Alice", "Bob") ORDER BY name')
balances = cursor.fetchall()
print('Final balances after savepoint rollback:', balances)

## Best Practices for Transactions

- **Keep transactions short**: Long-running transactions can cause locking issues
- **Use context managers**: Automatic cleanup on success/failure
- **Handle exceptions properly**: Always rollback on errors
- **Test transaction logic**: Ensure atomicity and consistency
- **Consider isolation levels**: Choose appropriate level for your needs
- **Use savepoints for complex operations**: Allow partial rollbacks
- **Avoid nested transactions**: Can be complex to manage

In [ ]:
# View transaction history
cursor.execute('''
    SELECT t.id, a1.name as from_name, a2.name as to_name, t.amount, t.timestamp
    FROM transactions t
    JOIN accounts a1 ON t.from_account = a1.id
    JOIN accounts a2 ON t.to_account = a2.id
    ORDER BY t.timestamp
''')
history = cursor.fetchall()

print('Transaction history:')
for tx in history:
    print(f'ID {tx[0]}: {tx[1]} -> {tx[2]}: ${tx[3]} at {tx[4]}')

# Final account summary
cursor.execute('SELECT name, balance FROM accounts ORDER BY balance DESC')
summary = cursor.fetchall()
print('\nFinal account summary:')
for account in summary:
    print(f'{account[0]}: ${account[1]}')

## Summary

In this notebook, you learned:

- What transactions are and why they're important (ACID properties)
- Auto-commit vs explicit transactions
- How to start, commit, and rollback transactions
- Using context managers for automatic transaction handling
- Transaction isolation levels
- Savepoints for partial rollbacks
- Practical examples like money transfers
- Best practices for transaction management

Transactions are essential for maintaining data consistency, especially when multiple related operations must succeed or fail together. Always use transactions when performing multiple related database operations.

In [ ]:
# Close the connection
conn.close()
print('Database connection closed')